## NEXT:  Do this before exporting 'clean' csv/json files to use for full EDA:
## ask M&M about Mapped Location field in df_bldg_apps and df_bldg_issued. Has address and lat/long. Best solution for mapping later in Power BI or Tableau?

## Load & Transform Data

- Load
- Transfom headers into all lower case and replace spaces and / with underscores
- Export new csv files to use in EDA

In [1]:
from shapely.geometry import Point  # May not need this until I do EDA, but importing now as a reminder.
import pandas as pd
import numpy as np
import geopandas as gpd   
import matplotlib.pyplot as plt  
import folium                   
from folium.plugins import MarkerCluster

## Read in .csv files

In [2]:
# Building Dept. Permit Applications

df_bldg_apps = pd.read_csv('../data/Building_Permit_Applications_2020_06_05.csv')
df_bldg_apps.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Mapped Location
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,03/11/2020,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,37205,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944..."
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,12/02/2019,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,37015,N OF BULL RUN RD W OF OLD HICKORY BLVD,CLAYTON HOMES #054,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36...."


In [3]:
# Building Dept. Permits Issued
# low_memory = False was added to remove a low-memory warning. Doing this prevents the
# system from trying to assign dtypes until after the full file has been read
# Resource: https://tinyurl.com/stackoverflow-low-memory

df_bldg_issued = pd.read_csv('../data/Building_Permits_Issued_2020_06_05.csv', low_memory=False)
df_bldg_issued.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Census Tract,Mapped Location
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,11/18/2019,12/09/2019,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,37218,LOT 49 CARRINGTON PLACE PH 5,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218"
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,03/12/2020,03/12/2020,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,37027,UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4,ACCESS & MOBILITY INC,CARR,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\..."


In [4]:
# Planning/Zoning Applications

df_planning_apps = pd.read_csv('../data/Planning_Department_Development_Applications_2020_06_05.csv')
df_planning_apps.head(2)

,Date Submitted,Application Type Description,MPC Case #,Ordinance #,Status,MPC Meeting Date,MPC Action,Project Name,Location,Reviewer,...,Applicant Address 2,Applicant City,Applicant State,Applicant ZIP,Council 3rd Reading Date,Council 3rd Reading Action,Council District,Latitude,Longitude,Mapped Location
0,04/01/2019,Subdivision (Final Plat),2019S-086-001,NaN,PENDING,06/11/2020,NaN,FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...,227 MARCIA AVE 37209,Joren Dunnavant,...,NaN,Nashville,TN,37203,NaN,NaN,20 (Mary Carolyn Roberts),36.143923,-86.868254,"(36.143922831000054, -86.86825400699996)"
1,11/27/2019,Specific Plan (Final Site Plan),2016SP-076-008,NaN,PENDING,01/16/2020,NaN,RED OAKS TOWNHOMES,0 DEW ST 37206,Abbie Rickoff,...,NaN,Nashville,TN,37204,NaN,NaN,06 (Brett Withers),36.165962,-86.753490,"(36.165961579000054, -86.75348957099999)"


## Read in .geojson file, change crs (coordinate reference system)

In [5]:
# Neighborhood Assoc boundaries GIS file, using geoPandas

df_na_boundaries = gpd.read_file('../data/Neighborhood Association Boundaries (GIS)_2020_06_03.geojson')
print(df_na_boundaries.crs)
df_na_boundaries.head(2)

epsg:4326


,name,geometry
0,Historic Buena Vista,"MULTIPOLYGON (((-86.79511 36.17576, -86.79403 ..."
1,Charlotte Park,"MULTIPOLYGON (((-86.87460 36.15758, -86.87317 ..."


In [6]:
df_na_boundaries.crs = "EPSG:4326"
print(df_na_boundaries.crs)

EPSG:4326


## Renaming columns in 3 dfs to remove spaces, "/", #, etc.

In [7]:
df_bldg_apps.columns = (df_bldg_apps.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_bldg_apps.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,zip,subdivision___lot,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,03/11/2020,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,37205,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944..."
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,12/02/2019,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,37015,N OF BULL RUN RD W OF OLD HICKORY BLVD,CLAYTON HOMES #054,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36...."


In [8]:
df_bldg_apps.mapped_location.unique()

array(['748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944, -86.879062)',
       '4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36.242681, -86.929594)',
       '4119 MURFREESBORO PIKE\nANTIOCH, TN 37013\n(36.032211, -86.594799)',
       ...,
       '6680 CHARLOTTE PIKE B-5\nNASHVILLE, TN 37209\n(36.136609, -86.883701)',
       '3805 CHARLOTTE AVE\nNASHVILLE, TN 37209\n(36.152561, -86.831473)',
       '5610A GRANNY WHITE PIKE\nBRENTWOOD, TN 37027\n(36.046438, -86.815953)'],
      dtype=object)

In [9]:
df_bldg_issued.columns = (df_bldg_issued.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_bldg_issued.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,zip,subdivision___lot,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,11/18/2019,12/09/2019,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,37218,LOT 49 CARRINGTON PLACE PH 5,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218"
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,03/12/2020,03/12/2020,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,37027,UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4,ACCESS & MOBILITY INC,CARR,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\..."


In [10]:
df_planning_apps.columns = (df_planning_apps.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_planning_apps.head(2)

,date_submitted,application_type_descr,mpc_case_number,ordinance_number,status,mpc_meeting_date,mpc_action,project_name,location,reviewer,...,applicant_address_2,applicant_city,applicant_state,applicant_zip,council_3rd_reading_date,council_3rd_reading_action,council_district,latitude,longitude,mapped_location
0,04/01/2019,Subdivision (Final Plat),2019S-086-001,NaN,PENDING,06/11/2020,NaN,FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...,227 MARCIA AVE 37209,Joren Dunnavant,...,NaN,Nashville,TN,37203,NaN,NaN,20 (Mary Carolyn Roberts),36.143923,-86.868254,"(36.143922831000054, -86.86825400699996)"
1,11/27/2019,Specific Plan (Final Site Plan),2016SP-076-008,NaN,PENDING,01/16/2020,NaN,RED OAKS TOWNHOMES,0 DEW ST 37206,Abbie Rickoff,...,NaN,Nashville,TN,37204,NaN,NaN,06 (Brett Withers),36.165962,-86.753490,"(36.165961579000054, -86.75348957099999)"


In [11]:
print(df_bldg_apps.columns)
print(df_bldg_issued.columns)
print(df_planning_apps.columns)

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision___lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'mapped_location'],
      dtype='object')
Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision___lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'census_tract', 'mapped_location'],
      dtype='object')
Index(['date_submitted', 'application_type_descr', 'mpc_case_number',
       'ordinance_number', 'status', 'mpc_meeting_date', 'mpc_action',
       'project_name', 'location', 'reviewer', 'reviewer_email', 'case_descr',
       'applicant', 'applicant_representative', 'appli

In [12]:
# To correct an issue with noo many underscores in subdivision_lot column name

df_bldg_apps = df_bldg_apps.rename(columns = {'subdivision___lot': 'subdivision_lot'})
df_bldg_apps.columns

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision_lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'mapped_location'],
      dtype='object')

In [13]:
# To correct an issue with noo many underscores in subdivision_lot column name

df_bldg_issued = df_bldg_issued.rename(columns = {'subdivision___lot': 'subdivision_lot'})
df_bldg_issued.columns

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision_lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'census_tract', 'mapped_location'],
      dtype='object')

In [14]:
# Confirming that all are DataFrames

print(type(df_bldg_apps))
print(type(df_bldg_issued))
print(type(df_planning_apps))
print(type(df_na_boundaries))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [15]:
# Confirming that all dtypes are as expected
# All values in date_issued are null - removing this column from df

print(df_bldg_apps.info())
print(df_bldg_apps.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         3106 non-null   object 
 1   permit_type_descr     3106 non-null   object 
 2   permit_subtype_descr  3106 non-null   object 
 3   parcel                3106 non-null   object 
 4   date_entered          3106 non-null   object 
 5   date_issued           0 non-null      float64
 6   construction_cost     1651 non-null   float64
 7   address               3106 non-null   object 
 8   city                  3106 non-null   object 
 9   state                 3106 non-null   object 
 10  zip                   3106 non-null   int64  
 11  subdivision_lot       3105 non-null   object 
 12  contact               3105 non-null   object 
 13  permit_type           3106 non-null   object 
 14  permit_subtype        3106 non-null   object 
 15  ivr_tracking_number  

In [16]:
# To drop date_issued from building applications df

df_bldg_apps = df_bldg_apps.drop(columns = ['date_issued'])

In [17]:
df_bldg_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         3106 non-null   object 
 1   permit_type_descr     3106 non-null   object 
 2   permit_subtype_descr  3106 non-null   object 
 3   parcel                3106 non-null   object 
 4   date_entered          3106 non-null   object 
 5   construction_cost     1651 non-null   float64
 6   address               3106 non-null   object 
 7   city                  3106 non-null   object 
 8   state                 3106 non-null   object 
 9   zip                   3106 non-null   int64  
 10  subdivision_lot       3105 non-null   object 
 11  contact               3105 non-null   object 
 12  permit_type           3106 non-null   object 
 13  permit_subtype        3106 non-null   object 
 14  ivr_tracking_number   3106 non-null   int64  
 15  purpose              

In [18]:
# checking dtypes, building permits issued

df_bldg_issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33909 entries, 0 to 33908
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         33909 non-null  object 
 1   permit_type_descr     33909 non-null  object 
 2   permit_subtype_descr  33909 non-null  object 
 3   parcel                33909 non-null  object 
 4   date_entered          33909 non-null  object 
 5   date_issued           33909 non-null  object 
 6   construction_cost     33899 non-null  float64
 7   address               33909 non-null  object 
 8   city                  33909 non-null  object 
 9   state                 33909 non-null  object 
 10  zip                   33909 non-null  int64  
 11  subdivision_lot       33909 non-null  object 
 12  contact               33908 non-null  object 
 13  permit_type           33909 non-null  object 
 14  permit_subtype        33909 non-null  object 
 15  ivr_tracking_number

In [19]:
# Checking nulls, building permits issued

df_bldg_issued.isnull().sum()

permit_number             0
permit_type_descr         0
permit_subtype_descr      0
parcel                    0
date_entered              0
date_issued               0
construction_cost        10
address                   0
city                      0
state                     0
zip                       0
subdivision_lot           0
contact                   1
permit_type               0
permit_subtype            0
ivr_tracking_number       0
purpose                 467
council_district         46
census_tract             43
mapped_location           0
dtype: int64

In [20]:
df_planning_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   date_submitted              521 non-null    object 
 1   application_type_descr      521 non-null    object 
 2   mpc_case_number             520 non-null    object 
 3   ordinance_number            157 non-null    object 
 4   status                      521 non-null    object 
 5   mpc_meeting_date            521 non-null    object 
 6   mpc_action                  235 non-null    object 
 7   project_name                417 non-null    object 
 8   location                    498 non-null    object 
 9   reviewer                    521 non-null    object 
 10  reviewer_email              465 non-null    object 
 11  case_descr                  503 non-null    object 
 12  applicant                   501 non-null    object 
 13  applicant_representative    497 non

In [21]:
print("BOUNDARIES INFO", df_na_boundaries.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      288 non-null    object  
 1   geometry  288 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 4.6+ KB
BOUNDARIES INFO None
